In [1]:
# General imports
import os
import sys
import numpy as np

# Gymnasium
import gymnasium as gym
from gymnasium.spaces.box import Box

# Sinergym
import sinergym
from sinergym.utils.rewards import ExpReward
from sinergym.utils.wrappers import NormalizeObservation

In [2]:
# Add the EnergyPlus path to the system path
sys.path.append('./EnergyPlus-23-1-0')
# Set the EnergyPlus path as an environment variable
os.environ['EPLUS_PATH'] = './EnergyPlus-23-1-0'

In [3]:
# Environment ID
id = "Eplus-5zone-hot-continuous-stochastic-v1"

# Weather
weather_files = ['USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw']
weather_variability = (1.0,0.0,0.001)

# Reward function
reward = ExpReward
reward_kwargs = {
    'temperature_variables': ['air_temperature'],
    'energy_variables': ['HVAC_electricity_demand_rate'],
    'range_comfort_winter': [20, 23],
    'range_comfort_summer': [23, 26],
    'energy_weight': 0.1
}

In [4]:
# Create the environment
env = gym.make(
    id=id,
    weather_files=weather_files,
    weather_variability=weather_variability,
    reward=reward,
    reward_kwargs=reward_kwargs
)

# Normalize the observation space
env =  NormalizeObservation(env) 

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-continuous-stochastic-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-5zone-hot-continuous-stochastic-v1-res5]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 0, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode:

In [5]:
env.unwrapped.info()


    #==================================================================================#
                                ENVIRONMENT NAME: 5zone-hot-continuous-stochastic-v1
    #==================================================================================#
    #----------------------------------------------------------------------------------#
                                ENVIRONMENT INFO:
    #----------------------------------------------------------------------------------#
    - Building file: /home/luigi/Documents/SmartEnergyOptimizationRL/.venv/lib/python3.10/site-packages/sinergym/data/buildings/5ZoneAutoDXVAV.epJSON
    - Zone names: ['PLENUM-1', 'SPACE1-1', 'SPACE2-1', 'SPACE3-1', 'SPACE4-1', 'SPACE5-1']
    - Weather file(s): ['USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw']
    - Current weather used: /home/luigi/Documents/SmartEnergyOptimizationRL/.venv/lib/python3.10/site-packages/sinergym/data/weather/USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw
    - Episodes executed: 

In [6]:
for i in range(1):
    obs, info = env.reset()
    rewards = []
    terminated = False
    current_month = 0
    while not terminated:
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print("\n\nOBSERVATION: ", obs)
            print("REWARD: ", reward)
            print("TERMINATED: ", terminated)
            print("TRUNCATED: ", truncated)
            print("INFO: ", info)
        break

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-continuous-stochastic-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-5zone-hot-continuous-stochastic-v1-res5/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-5zone-hot-continuous-stochastic-v1-res5/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-5zone-hot-continuous-stochastic-v1-res5/Eplus-env-sub_run1/USA_AZ_Da

In [7]:
env.close()

Progress: |***************************************************************************************************| 99%


[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-continuous-stochastic-v1]
